In [31]:
%matplotlib inline
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import ApproxTimeEvolution
import pandas as pd

![problem graph](./assets/graph.png)

In [5]:
"""
  A = c(0, 5.8, 5.8, 7.6, 8.8)
  B = c(5.3, 0, 2.4, 4.3, 4.3)
  C = c(7.1, 1.7, 0, 3.5, 4.7)
  D = c(8.7, 4.0, 3.4, 0, 1.4)
  E = c(9.8, 5.1, 4.5, 1.4, 0)
"""

idx = pd.Index(['A', 'B', 'C', 'D', 'E'])

df = pd.DataFrame([
    [0, 5.8, 5.8, 7.6, 8.8],
    [5.3, 0, 2.4, 4.3, 4.3],
    [7.1, 1.7, 0, 3.5, 4.7],
    [8.7, 4.0, 3.4, 0, 1.4],
    [9.8, 5.1, 4.5, 1.4, 0]
], columns=['A', 'B', 'C', 'D', 'E'], index=idx)
df

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.3,0.0,2.4,4.3,4.3
C,7.1,1.7,0.0,3.5,4.7
D,8.7,4.0,3.4,0.0,1.4
E,9.8,5.1,4.5,1.4,0.0


In [7]:
total_qubits = df.shape[0]
print(f"total used qubits: {total_qubits}")

total used qubits: 5


In [8]:
H = np.zeros((2 ** total_qubits, 2 ** total_qubits))
H

tensor([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], requires_grad=True)

In [37]:
def tsp_hamiltonian(distances):
    num_cities = len(distances)
    H = np.zeros((2 ** num_cities, 2 ** num_cities))

    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                for k in range(num_cities):
                    H += distances[i, j] * np.kron(np.kron(np.eye(2 ** k), np.outer([1, 0], [0, -1])), np.eye(2 ** (num_cities - k - 1)))
    return qml.Hamiltonian(H, [qml.PauliZ(i) for i in range(num_cities)])

In [38]:
dev = qml.device('default.qubit', wires=total_qubits)

@qml.qnode(dev)
def circuit(data, time):
    ApproxTimeEvolution(tsp_hamiltonian(data), time, 1)
    return qml.expval(qml.PauliZ(0))

In [33]:
circuit(df.to_numpy(),1)

ValueError: hamiltonian must be of type pennylane.Hamiltonian, got tensor